<a href="https://colab.research.google.com/github/pkolachi/lexicalnormalization/blob/master/exptnbs/LexicalNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup and Configuration

In [ ]:
!git clone https://github.com/pkolachi/lexicalnormalization

Cloning into 'multilexnorm'...
remote: Counting objects: 43, done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 1), reused 0 (delta 0)
Unpacking objects: 100% (43/43), done.


In [ ]:
LANGS = {'da': 'Danish',
         'en': 'English',
         'es': 'Spanish',
         'fr': 'French',
         'hr': 'Croatian',
         'iden': 'Indonesian-English',
         'it': 'Italian',
         'nl': 'Dutch',
         'sl': 'Slovenian',
         'sr': 'Serbian',
         'tr': 'Turkish',
         'trde': 'Turkish-German',
         }
SMPLS = ['en']

### Load Data

In [ ]:
from collections import defaultdict 
from operator import itemgetter
import os.path 

def read_data(inpfile):
  with open(inpfile) as ins:
    sent = []
    for lne in ins:
      if not lne.strip():
        yield sent
        sent = []
      else:
        sent.append(lne.split('\t', 1))

DATA = defaultdict(lambda: defaultdict(list))
for lang in SMPLS:
  datadir = os.path.join('multilexnorm', 'data', lang)
  trnfile = os.path.join(datadir, 'train.norm')
  devfile = os.path.join(datadir, 'dev.norm')
  tstfile = os.path.join(datadir, 'test.norm')
  if os.path.isdir(datadir) and os.path.isfile(trnfile):
    crp = list(read_data(trnfile))
    X = list(map(itemgetter(0), crp))  # load input (un-normalized sentences)
    Y = list(map(itemgetter(1), crp))  # load output (normalized sentences)
    DATA[lang]['fulltrn'] = (X, Y)
  if os.path.isdir(datadir) and os.path.isfile(devfile):
    crp = list(read_data(devfile))
    DATA[lang]['dev'] = (list(map(itemgetter(0), crp)), 
                         list(map(itemgetter(1), crp)))
  if os.path.isdir(datadir) and os.path.isfile(tstfile):
    crp = list(read_data(tstfile))
    DATA[lang]['tst'] = (list(map(itemgetter(0), crp)), 
                         list(map(itemgetter(1), crp)))

In [ ]:
TST_RATIO = 0.25
from sklearn.model_selection import train_test_split 

for lang in SMPLS:
  if 'fulltrn' in DATA[lang]:
    trn_x, hld_x, trn_y, hld_y = train_test_split(DATA[lang]['fulltrn'][0], 
                                                  DATA[lang]['fulltrn'][1], 
                                                  test_size=TST_RATIO, 
                                                  random_state=0, 
                                                  shuffle=False)
    DATA[lang]['trn'] = (trn_x, trn_y)
    DATA[lang]['hld'] = (hld_x, hld_y)

### Preprocessing